# Classification Analysis
This notebook will contain classification analysis for both the sensed and pipelined algorithms. Analysis will be preformed in regards for the sensed and pipelined algorthms themselves, as well as the ensemble algorithms. The analysis for the ensemble algorithm will focus on the HAMF android phones and the HAHF iOS phones.

## Dependencies

In [ ]:
# for reading and validating data
import emeval.input.spec_details as eisd
import emeval.input.phone_view as eipv
import emeval.input.eval_view as eiev

In [ ]:
import emeval.viz.phone_view as ezpv
import emeval.viz.eval_view as ezev
import emeval.viz.geojson as ezgj

In [ ]:
# for analysized view
import emeval.analysed.phone_view as eapv

In [ ]:
import emeval.metrics.segmentation as ems

In [ ]:
# Metrics helpers
import emeval.metrics.dist_calculations as emd
import emeval.metrics.reference_trajectory as emr

In [ ]:
import pandas as pd
pd.options.display.float_format = '{:.6f}'.format
import arrow

In [ ]:
import numpy as np
import scipy as sp
import scipy.interpolate
import scipy.integrate

In [ ]:
# For plots
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# For maps
import geopandas as gpd
import shapely as shp
import folium
import branca.element as bre

In [ ]:
# For easier debugging while working on modules
import importlib

In [ ]:
import arrow

In [ ]:
def pv_to_pkl(pv, pkl_file_name):
    if type(pv) is not list: pv = [pv]
    import pickle
    with open(pkl_file_name, 'wb') as outp:
        for pv in pv_l:
            pickle.dump(pv, outp, pickle.HIGHEST_PROTOCOL)

In [ ]:
def import_sd_and_pv_from_server(trips  = ["unimodal_trip_car_bike_mtv_la", "car_scooter_brex_san_jose", "train_bus_ebike_mtv_ucb"], 
                                 AUTHOR_EMAIL  = "shankari@eecs.berkeley.edu", 
                                 DATASTORE_LOC = "http://localhost:8080", 
                                 pkl_file_name = None):
    sd_l = []
    pv_l = []
    for trip in trips:
        sd = eisd.ServerSpecDetails(DATASTORE_LOC, AUTHOR_EMAIL, trip)
        pv = eipv.PhoneView(sd)
        sd_l.append(sd)
        pv_l.append(pv)
    if pkl_file_name:
        import pickle
        with open(pkl_file_name, 'wb') as outp:
            for pv in pv_l:
                pickle.dump(pv, outp, pickle.HIGHEST_PROTOCOL)
    return sd_l, pv_l

In [ ]:
def import_pv_from_pkl(pkl_file_name, 
                       trips = ["unimodal_trip_car_bike_mtv_la", "car_scooter_brex_san_jose", "train_bus_ebike_mtv_ucb"]):
    import pickle
    pv_l = []
    with open('pv.pkl', 'rb') as inp:
        for trip in trips:
            pv_l.append(pickle.load(inp))
    return pv_l

In [ ]:
(pv_la, pv_sj, pv_ucb) = import_pv_from_pkl('pv.pkl')

### Get the sensed data for each trip

In [ ]:
%%capture
ems.fill_sensed_section_ranges(pv_la)
ems.fill_sensed_section_ranges(pv_sj)
ems.fill_sensed_section_ranges(pv_ucb)

## Get sensed timeline

In [ ]:
def get_trip_ss_and_gts_timeline(pv, os, role):
    assert os in ['android', 'ios'], 'UNKNOWN OS'
    assert role in ['accuracy_control', 'HAHFDC', 'HAMFDC', 'MAHFDC', 'power_control'], "UNKNOWN ROLE"
    trips = []
    for phone_os, phone_map in pv.map().items():
        if os != phone_os:
            continue
        for phone_label, phone_detail_map in phone_map.items():
            if "control" in phone_detail_map["role"]:
                continue
            for r in phone_detail_map["evaluation_ranges"]:
                if r['eval_role_base'] != role:
                    continue
                for tr in r["evaluation_trip_ranges"]:
                    tr_ss  = []
                    tr_gts = []
                    for ss in tr["sensed_section_ranges"]:
                        tr_ss.append(ss)
                    for section in tr["evaluation_section_ranges"]:
                        section_gt_leg = pv.spec_details.get_ground_truth_for_leg(tr['trip_id_base'],
                                                                                  section['trip_id_base'],
                                                                                  tr['start_ts'],
                                                                                  tr['end_ts'])
                        
                        if section_gt_leg["type"] == "WAITING":
                            continue
                        gts = {'start_ts': section['start_ts'], 
                               'end_ts': section['end_ts'], 
                               'mode': section_gt_leg['mode']}
                        tr_gts.append(gts)
                    # now, we build a timeline for each trip
                    trip = tr.copy()
                    trip['ss_timeline']  = tr_ss
                    trip['gts_timeline'] = tr_gts
                    trips.append(trip)
    return trips

## Binary Classification (in seconds)

#### rab base mode map

In [ ]:
RBMM = {     "WALKING": "WALKING",
             "RUNNING" : "WALKING", 
             "CYCLING" : "CYCLING",
             "BICYCLING": "CYCLING",
             "ESCOOTER": "CYCLING", 
             "AUTOMOTIVE" : "AUTOMOTIVE",
             "BUS": "AUTOMOTIVE",
             "TRAIN": "AUTOMOTIVE",
             "LIGHT_RAIL": "AUTOMOTIVE",
             "SUBWAY": "AUTOMOTIVE",
             "CAR": "AUTOMOTIVE",
             "AIR_OR_HSR": "AUTOMOTIVE",
             "INVALID" : "INVALID"}

#### cleaned base mode map

e-mission-server.emission.core.wrapper.motionactivity.py

In [ ]:
CBMM = {0 : 'AUTOMOTIVE', 
        1 : 'CYCLING', 
        2 : 'WALKING', 
        3 : 'WALKING', 
        4 : 'UNKNOWN', 
        5 : 'WALKING', 
        7 : 'WALKING', 
        8 : 'WALKING', 
        9 : 'UNKNOWN', 
        10 : 'AUTOMOTIVE', 
        11 : 'AIR_OR_HSR', 
         "WALKING": "WALKING",
         "RUNNING" : "WALKING", 
         "CYCLING" : "CYCLING",
         "BICYCLING": "CYCLING",
         "ESCOOTER": "CYCLING", 
         "AUTOMOTIVE" : "AUTOMOTIVE",
         "BUS": "AUTOMOTIVE",
         "TRAIN": "AUTOMOTIVE",
         "LIGHT_RAIL": "AUTOMOTIVE",
         "SUBWAY": "AUTOMOTIVE",
         "CAR": "AUTOMOTIVE",
         "AIR_OR_HSR": "AIR_OR_HSR",
         "INVALID" : "INVALID", 
         "UNKNOWN" : "UNKNOWN"}

#### inferred base mode map

e-mission-server.emission.core.wrapper.modeprediction.py

In [ ]:
IBMM = {0 : 'UNKNOWN',    # UNKNOWN
        1 : 'WALKING',    # WALKING
        2 : 'CYCLING',    # BICYCLING
        3 : 'AUTOMOTIVE', # BUS
        4 : 'TRAIN',      # TRAIN
        5 : 'AUTOMOTIVE', # CAR
        6 : 'TRAIN',      # AIR_OR_HSR
        7 : 'TRAIN',      # SUBWAY
        8 : 'TRAIN',      # TRAM
        9 : 'TRAIN',      # LIGHT_RAIL
         "WALKING": "WALKING",
         "RUNNING" : "WALKING", 
         "CYCLING" : "CYCLING",
         "BICYCLING": "CYCLING",
         "ESCOOTER": "CYCLING", 
         "AUTOMOTIVE" : "AUTOMOTIVE",
         "BUS": "AUTOMOTIVE",
         "TRAIN": "TRAIN",
         "LIGHT_RAIL": "TRAIN",
         "SUBWAY": "TRAIN",
         "CAR": "AUTOMOTIVE",
         "AIR_OR_HSR": "TRAIN",
         "INVALID" : "INVALID", 
         "UNKNOWN" : "UNKNOWN"}

In [ ]:
def get_binary_class_in_sec(os, role, pv, BASE_MODE, test=False, test_trip=None):
    if not test:
        if type(pv) is not list: pv = [pv]
        trips = []
        for v in pv:
            trips.extend(get_trip_ss_and_gts_timeline(v, os, role))
    else:
        trips = test_trip if type(test_trip) is list else [test_trip]
    TP, FN, FP, TN = {}, {}, {}, {}
    for trip in trips:
        for mode in set(BASE_MODE.values()):
            # get no sensed for a given mode 
            #### Note, this relies on the assumption that our timelines are always ordered
            if len(trip['ss_timeline']) > 0 and len(trip['gts_timeline']) > 0:
                no_sensed_start = 0
                no_sensed_end = 0
                try:
                    no_sensed_start = max(0, trip['ss_timeline'][0]['start_ts'] - trip['gts_timeline'][0]['start_ts']) 
                    no_sensed_end = max(0, trip['gts_timeline'][-1]['end_ts'] - trip['ss_timeline'][-1]['end_ts'])
                except:
                    no_sensed_start = max(0, trip['ss_timeline'][0]['data']['start_ts'] - trip['gts_timeline'][0]['start_ts'])
                    no_sensed_end += max(0, trip['gts_timeline'][-1]['end_ts'] - trip['ss_timeline'][-1]['data']['end_ts'])
                if mode == 'INVALID':
                    FP[mode] = no_sensed_end + no_sensed_start
                elif mode == BASE_MODE[trip['gts_timeline'][0]['mode']] and mode == BASE_MODE[trip['gts_timeline'][-1]['mode']]:
                    FN[mode] = FN.setdefault(mode, 0) + no_sensed_start + no_sensed_end
                elif mode == BASE_MODE[trip['gts_timeline'][0]['mode']]:
                    FN[mode] = FN.setdefault(mode, 0) + no_sensed_start
                    TN[mode] = TN.setdefault(mode, 0) + no_sensed_end
                elif mode == BASE_MODE[trip['gts_timeline'][-1]['mode']]:
                    FN[mode] = FN.setdefault(mode, 0) + no_sensed_end
                    TN[mode] = TN.setdefault(mode, 0) + no_sensed_start
                else:
                    TN[mode] = TN.setdefault(mode, 0) + no_sensed_start + no_sensed_end
#                 if mode == 'INVALID':
#                     FP[mode] = no_sensed_end + no_sensed_start
#                 elif mode == BASE_MODE[trip['gts_timeline'][0]['mode']]:
#                     # False Negative
#                     FN[mode] = FN.setdefault(mode, 0) + no_sensed_start
#                 elif mode == BASE_MODE[trip['gts_timeline'][-1]['mode']]:
#                     FN[mode] = FN.setdefault(mode, 0) + no_sensed_end
#                 else:
#                     TN[mode] = TN.setdefault(mode, 0) + no_sensed_start
            for ss in trip['ss_timeline']:
#                 print(ss.keys())
                if 'data' in ss.keys():
                    ss = ss['data']
                    ss['mode'] = ss['sensed_mode']
                ss_dur = ss['end_ts'] - ss['start_ts']
                gts_dur = 0
                for gts in trip['gts_timeline']:
                    if ss['end_ts'] >= gts['start_ts'] and ss['start_ts'] <= gts['end_ts']:
                        dur = min(ss['end_ts'], gts['end_ts']) - max(ss['start_ts'], gts['start_ts'])
                        gts_dur += dur
                        if BASE_MODE[mode] == BASE_MODE[ss['mode']] and BASE_MODE[mode] == BASE_MODE[gts['mode']]:
                            TP[mode] = TP.setdefault(mode, 0) + dur
                        elif BASE_MODE[mode] == BASE_MODE[ss['mode']] and BASE_MODE[mode] != BASE_MODE[gts['mode']]:
                            FP[mode] = FP.setdefault(mode, 0) + dur
                        elif BASE_MODE[mode] != BASE_MODE[ss['mode']] and BASE_MODE[mode] == BASE_MODE[gts['mode']]:
                            FN[mode] = FN.setdefault(mode, 0) + dur
                        else:
                            TN[mode] = TN.setdefault(mode, 0) + dur
                leftover = ss_dur - gts_dur
                assert leftover >= 0, f"ERROR, NEGATIVE LEFTOVER OF {leftover}, NEED TO INVESTIGATE"
                if leftover > 0:
                    # invalid base mode maps to NO_GT mode
                    if mode == 'INVALID':
                        FN[mode] = TP.setdefault(mode, 0) + leftover
                    # We have no gts, but our modes are equal, so a false positive
                    elif BASE_MODE[mode] == BASE_MODE[ss['mode']]:
                        FP[mode] = FP.setdefault(mode, 0) + leftover
                    # We have no_gts, but our modes are unequal, so a true negative
                    else:
                        TN[mode] = TN.setdefault(mode, 0) + leftover
    return TP, FP, FN, TN

# $F_\beta$ score
$$
F_\beta = \frac {(1 + \beta^2) \cdot \mathrm{true\ positive} }{(1 + \beta^2) \cdot \mathrm{true\ positive} + \beta^2 \cdot \mathrm{false\ negative} + \mathrm{false\ positive}}
$$

In [ ]:
def get_F_score(os, role, pv, BASE_MODE, beta=1, test=False, test_trip=None):
    if not test:
        assert os in ['android', 'ios'], 'UNKNOWN OS'
        assert role in ['accuracy_control', 'HAHFDC', 'HAMFDC', 'MAHFDC', 'power_control'], "UNKNOWN ROLE"
        (TP, FP, FN, TN) = get_binary_class_in_sec(os, role, pv, BASE_MODE)
    else:
        (TP, FP, FN, TN) = get_binary_class_in_sec(os, role, pv, BASE_MODE, test=True, test_trip=test_trip)
    F_score = {}
    for mode in TP.keys():
        numerator   = (1 + beta**2) * TP.setdefault(mode, 0)
        denominator = (1+beta**2) * TP.setdefault(mode, 0) + beta**2*FN.setdefault(mode, 0) + FP.setdefault(mode, 0)
        F_score[mode] = (numerator)/(denominator)
    # initializing K 
    K = 10
    for key in F_score:

        # rounding to K using round()
        F_score[key] = round(F_score[key], K)
    return F_score

In [ ]:
def display_f_score(os, pv, BASE_MODE):
    return (pd.DataFrame([ (k, round(get_F_score(os, 'HAHFDC', pv, BASE_MODE, beta=1)[k], 4)) for k in get_F_score(os, 'HAHFDC', pv, BASE_MODE, beta=1)]).round(4),
            pd.DataFrame([ (k, round(get_F_score(os, 'HAMFDC', pv, BASE_MODE, beta=1)[k], 4)) for k in get_F_score(os, 'HAMFDC', pv, BASE_MODE, beta=1)]).round(4),
            pd.DataFrame([ (k, round(get_F_score(os, 'MAHFDC', pv, BASE_MODE, beta=1)[k], 4)) for k in get_F_score(os, 'MAHFDC', pv, BASE_MODE, beta=1)]).round(4)
            )

## Confusion Matrix
We will now generate confusion matrices based off OS and role, with the acctual modes as the rows, the predicted modes as the columns, and the entries as the base unit for the duration measurement

#### cleaned index map

In [ ]:
CIM = {0 : 'IN_VEHICLE', 
       1 : 'BICYCLING', 
       2 : 'ON_FOOT', 
       3 : 'STILL', 
       4 : 'UNKNOWN', 
       5 : 'TILTING', 
       7 : 'WALKING', 
       8 : 'RUNNING', 
       9 : 'NONE', 
       10 : 'STOPPED_WHILE_IN_VEHICLE', 
       11 : 'AIR_OR_HSR'}

#### inferred index map

In [ ]:
IIM = {
    0 : 'UNKNOWN', 
    1 : 'WALKING', 
    2 : 'BICYCLING', 
    3 : 'BUS', 
    4 : 'TRAIN', 
    5 : 'CAR', 
    6 : 'AIR_OR_HSR',
    7 : 'SUBWAY',
    8 : 'TRAM',
    9 : 'LIGHT_RAIL'
}

In [ ]:
def get_confusion_matrix(os, role, pv, test=False, test_trip=None):
    cm_l = []
    if not test:
        assert os in ['android', 'ios'], 'UNKNOWN OS'
        assert role in ['accuracy_control', 'HAHFDC', 'HAMFDC', 'MAHFDC', 'power_control'], "UNKNOWN ROLE"
        if type(pv) is not list:
            pv = [pv]
        trips = []
        for v in pv :
            trips.extend(get_trip_ss_and_gts_timeline(v, os, role))
    else:
        trips = test_trip if type(test_trip) is list else [test_trip]
    for trip in trips:
        ## get no sensed
        if len(trip['ss_timeline']) > 0 and len(trip['gts_timeline']) > 0:
            no_sensed_start = 0
            no_sensed_end = 0
            try:
                no_sensed_start = max(0, trip['ss_timeline'][0]['start_ts'] - trip['gts_timeline'][0]['start_ts']) 
                no_sensed_end = max(0, trip['gts_timeline'][-1]['end_ts'] - trip['ss_timeline'][-1]['end_ts'])
            except:
                no_sensed_start = max(0, trip['ss_timeline'][0]['data']['start_ts'] - trip['gts_timeline'][0]['start_ts'])
                no_sensed_end += max(0, trip['gts_timeline'][-1]['end_ts'] - trip['ss_timeline'][-1]['data']['end_ts'])
            if no_sensed_start:
                ns = {'sensed_mode' : 'NO_SENSED', trip['gts_timeline'][0]['mode'] : no_sensed_start}
                cm_l.append(ns)
            if no_sensed_end:
                ns = {'sensed_mode' : 'NO_SENSED', trip['gts_timeline'][-1]['mode'] : no_sensed_end}
                cm_l.append(ns)
        for ss in trip['ss_timeline']:
            if 'data' in ss.keys():
                ss = ss['data']
                ss['mode'] = ss['sensed_mode']
            ss_dur = ss['end_ts'] - ss['start_ts']
            gts_dur = 0
            cm = {}
            for gts in trip['gts_timeline']:
                dur = 0
                if ss['end_ts'] >= gts['start_ts'] and ss['start_ts'] <= gts['end_ts']:
                    dur = min(ss['end_ts'], gts['end_ts']) - max(ss['start_ts'], gts['start_ts'])
                    
                gts_dur += dur
                cm[gts['mode']] = cm.setdefault(gts['mode'], 0) + dur
            leftover = ss_dur - gts_dur
            assert leftover >= 0, f"ERROR, NEGATIVE LEFTOVER OF {leftover}, NEED TO INVESTIGATE"
            cm['NO_GT'] = cm.setdefault('NO_GT', 0) + leftover
            cm['sensed_mode'] = ss['mode']
            
            cm_l.append(cm)
    return cm_l

In [ ]:
def plot_cm(os, pv, d_type, INDEX_MAP=None):
    fig, ax = plt.subplots(1,3, figsize=(15,15), dpi=300, sharey=True)
    y=.95
    fig.text(0.5, 0.04, 'Predicted Label', ha='center', fontsize='xx-large')
    fig.text(0.04, 0.5, 'True Label', va='center', rotation='vertical', fontsize='xx-large')
    for k, role in enumerate(["HAHFDC", "HAMFDC", "MAHFDC"]):
        if d_type =='raw':
            title = f"Confusion Matrices for Raw Output Data on Phones Running {os} \n by Calibration Settings"
            df = pd.DataFrame(get_confusion_matrix(os, role, pv)).groupby('sensed_mode').sum()
            fname = f"images/raw_cm_{os}"
        elif d_type == 'clean':
            title = f"Confusion Matrices for Clean Output Data on Phones Running {os} \n by Calibration Settings"
            df = pd.DataFrame(get_confusion_matrix(os, role, pv)).groupby('sensed_mode').sum().rename(index=INDEX_MAP)
            fname = f"images/clean_cm_{os}"
        elif d_type == 'random_forest' or 'gis':
            df = pd.DataFrame(get_confusion_matrix(os, role, pv)).groupby('sensed_mode').sum().rename(index=INDEX_MAP)
            fname = f"images/{d_type}_cm_{os}"
            if d_type == 'random_forest':
                title = f"Confusion Matrices for Inferred Output Data (Random Forest) on Phones Running {os} \n by Calibration Settings"
            else:
                title = f"Confusion Matrices for Inferred Output Data (GIS) on Phones Running {os} \n by Calibration Settings"
        else:
            assert 0, f'INVALID d_type {d_type}'
        df = df.reindex(columns=['WALKING', 'BICYCLING', 'ESCOOTER', 'CAR', 'BUS', 'SUBWAY', 'LIGHT_RAIL', 'TRAIN', 'NO_GT'])
#         df = df.div(df.sum(axis=1), axis=0)
        cm = ax[k].imshow(df.transpose(), interpolation='nearest',  cmap=plt.cm.coolwarm, aspect='auto')
        ax[k].set_title(role)
    #     plt.colorbar(cm, ax=ax[0])
        tick_marks = np.arange(len(df))
        ax[k].set_yticks(np.arange(len(df.columns)))
        ax[k].set_xticks(np.arange(len(df)))
        ax[k].set_yticklabels(df)
        ax[k].set_xticklabels(df.index, rotation=80)
        color_thresh = df.max().max() / 4
        for i, j in itertools.product(range(df.shape[1]), range(df.shape[0])  ):
            ax[k].text(j, i, (int(df.transpose().iat[i,j])), horizontalalignment='center', 
                   color='white' 
                       if df.transpose().iat[i,j] < color_thresh 
#                            or df.transpose().iat[i,j] in df.max()
                       else 'black')
    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
    fig.colorbar(cm, cax=cbar_ax)
    plt.suptitle(title, weight='bold', size='x-large', y=y)
    plt.savefig(fname=fname)

## Analyzed Data

#### cleaned view

In [ ]:
cv_la   = eapv.create_analysed_view(pv_la, "http://localhost:8080", "analysis/recreated_location", "analysis/cleaned_trip", "analysis/cleaned_section")
cv_sj   = eapv.create_analysed_view(pv_sj, "http://localhost:8080", "analysis/recreated_location", "analysis/cleaned_trip", "analysis/cleaned_section")
cv_ucb  = eapv.create_analysed_view(pv_ucb, "http://localhost:8080", "analysis/recreated_location", "analysis/cleaned_trip", "analysis/cleaned_section")

#### inferred view random forest

In [ ]:
rfv_la   = eapv.create_analysed_view(pv_la, "http://localhost:8080", "analysis/recreated_location", "analysis/cleaned_trip", "analysis/inferred_section")
rfv_sj   = eapv.create_analysed_view(pv_sj, "http://localhost:8080", "analysis/recreated_location", "analysis/cleaned_trip", "analysis/inferred_section")
rfv_ucb  = eapv.create_analysed_view(pv_ucb, "http://localhost:8080", "analysis/recreated_location", "analysis/cleaned_trip", "analysis/inferred_section")

#### inferred view GIS

In [ ]:
gisv_la   = eapv.create_analysed_view(pv_la, "http://localhost:8080", "analysis/recreated_location", "analysis/cleaned_trip", "analysis/inferred_section")
gisv_sj   = eapv.create_analysed_view(pv_sj, "http://localhost:8080", "analysis/recreated_location", "analysis/cleaned_trip", "analysis/inferred_section")
gisv_ucb  = eapv.create_analysed_view(pv_ucb, "http://localhost:8080", "analysis/recreated_location", "analysis/cleaned_trip", "analysis/inferred_section")

# Results 

#### Raw data

In [ ]:
display_f_score('android', [pv_la, pv_sj, pv_ucb], RBMM)

In [ ]:
display_f_score('ios', [pv_la, pv_sj, pv_ucb], RBMM)

In [ ]:
plot_cm('ios', [pv_la, pv_sj, pv_ucb], 'raw')

In [ ]:
plot_cm('android', [pv_la, pv_sj, pv_ucb], 'raw')

#### Cleaned data

In [ ]:
display_f_score('android', [cv_la, cv_sj, cv_ucb], CBMM)

In [ ]:
display_f_score('ios', [cv_la, cv_sj, cv_ucb], CBMM)

In [ ]:
plot_cm('ios', [cv_la, cv_sj, cv_ucb], 'clean', CIM)

In [ ]:
plot_cm('android', [cv_la, cv_sj, cv_ucb], 'clean', CIM)

#### Random Forrest

In [ ]:
display_f_score('android', [rfv_la, rfv_sj, rfv_ucb], IBMM)

In [ ]:
display_f_score('ios', [rfv_la, rfv_sj, rfv_ucb], IBMM)

In [ ]:
plot_cm('ios', [rfv_la,rfv_sj,rfv_ucb], 'random_forest', INDEX_MAP=IIM)

In [ ]:
plot_cm('android', [rfv_la,rfv_sj,rfv_ucb], 'random_forest', INDEX_MAP=IIM)

#### GIS

In [ ]:
display_f_score('android', [gisv_la, gisv_sj, gisv_ucb], IBMM)

In [ ]:
display_f_score('ios', [gisv_la, gisv_sj, gisv_ucb], IBMM)

In [ ]:
plot_cm('ios', [gisv_la,gisv_sj,gisv_ucb], 'gis', INDEX_MAP=IIM)

In [ ]:
plot_cm('android', [gisv_la,gisv_sj,gisv_ucb], 'gis', INDEX_MAP=IIM)

# Unite Testing

* get_binary_class_in_sec(os, role, pv, BASE_MODE, test=False, test_trip=None)
* get_F_score(os, role, pv, BASE_MODE, beta=1, test=False, test_trip=None)
* get_confusion_matrix(os, role, pv, test=False, test_trip=None)

## Example timelines

### No sensed at the beggining, No GT at the end, Multimodal


* pipeline timeline

WALKING (0.5, 1.35) -> CYCLING (1.35, 2.7) -> WALKING (2.7, 3.5)

* ground truth timeline

WALKING (0, 1.05) -> BICYCLING (1.05, 3.1)

#### binary classifiers
    + TP 
        + [WALKING: 0.55] 
        + [CYCLING: 1.35]
    + FP 
        + [WALKING: 1.10] 
        + [INVALID: 0.5]
    + FN 
        + [WALKING: 0.50] 
        + [CYCLING: 0.70]
        + [INVALID: 0.4]
    + TN 
        + [WALKING: 1.35] 
        + [CYCLING: 1.45]
        + [INVALID: 2.60]
* $F_1$ score
    + [WALKING: 0.41]
    + [CYCLING: 0.79]
    + [INVALID: ....]
    
#### Confusion Matrix


sensed = [WALKING, CYCLING, NO_SENSED]

ground truth = [WALKING, BICYCLING, NO_GT]

origin top left

    [0.55, 0.00, 0.50]
    [0.70, 1.35, 0.00]
    [0.40, 0.00, 0.00]

In [ ]:
test_trip = {
    'ss_timeline'  : [{'start_ts' : 0.5, 'end_ts' : 1.35, 'mode' : 'WALKING'},
                      {'start_ts' : 1.35, 'end_ts' : 2.7, 'mode' : 'CYCLING'}, 
                      {'start_ts' : 2.7, 'end_ts' : 3.5, 'mode' : 'WALKING'}],
    'gts_timeline' : [{'start_ts' : 0, 'end_ts' : 1.05, 'mode' : 'WALKING'}, 
                      {'start_ts' : 1.05, 'end_ts' : 3.1, 'mode' : 'BICYCLING'}]
}
test_BMM = {'WALKING' : 'WALKING', 'CYCLING' : 'CYCLING', 'BICYCLING' : 'CYCLING', 'NO_SENSED' : 'INVALID', 'NO_GT' : 'INVALID', 'INVALID' : 'INVALID'}

#### get_binary_class_in_sec

In [ ]:
res = get_binary_class_in_sec(..., ..., ..., test_BMM, test=True, test_trip=test_trip)

In [ ]:
assert round(res[0]['WALKING'], 2) == 0.55 and round(res[0]['CYCLING'], 2) == 1.35 and res[0]['INVALID'] == 0 and len(res[0]) == 3, f"TP wrong"
assert round(res[1]['WALKING'], 2) == 1.10 and round(res[1]['INVALID'], 2) == 0.50 and len(res[1]) == 2, f"FP wrong"
assert round(res[2]['WALKING'], 2) == 0.50 and round(res[2]['CYCLING'], 2) == 0.70 and round(res[2]['INVALID'], 2) == 0.40 and len(res[2]) == 3, f"FN wrong"
assert round(res[3]['WALKING'], 2) == 1.35 and round(res[3]['CYCLING'], 2) == 1.45 and round(res[3]['INVALID'], 2) == 2.60 and len(res[2]) == 3, f"TN wrong"

#### get_F_score

In [ ]:
res = get_F_score(..., ..., ..., test_BMM, beta=1, test=True, test_trip=test_trip)
assert round(res['WALKING'],2) == 0.41 and round(res['CYCLING'], 2) == 0.79 and res['INVALID'] == 0, f"F_1 scores wrong"

#### get_confusion_matrix

In [ ]:
res = get_confusion_matrix(..., ..., ..., test=True, test_trip=test_trip)
df = pd.DataFrame(res).groupby('sensed_mode').sum()
assert 'WALKING' in df.index and 'CYCLING' in df.index and 'NO_SENSED' in df.index, f"INCORRECT INDECIES \n EXPECTED \t 'WALKING', 'CYCLING', 'NO_SENSED' \n GOT \t \t {df.index}"
assert df.loc['WALKING'].loc['WALKING'] == 0.55 and df.loc['WALKING'].loc['BICYCLING'] == 0.70 and round(df.loc['WALKING'].loc['NO_GT'],2) == 0.40
assert df.loc['CYCLING'].loc['WALKING'] == 0 and df.loc['CYCLING'].loc['BICYCLING'] == 1.35 and round(df.loc['CYCLING'].loc['NO_GT'],2) == 0.0
assert round(df.loc['NO_SENSED'].loc['WALKING'], 2) == 0.50 and df.loc['NO_SENSED'].loc['BICYCLING'] == 0.0 and round(df.loc['NO_SENSED'].loc['NO_GT'],2) == 0.0

### No sensed at beggining and end, multimodal

* pipeline timeline
WALKING (0.5, 1) -> BIKING (1, 1.5)

* ground truth timeline
WALKING (0, 2)

#### Multiclass classifiers
* TP
    + WALKING [0.5]
* FP
    + CYCLING [0.5]
    + INVALID [1]
* FN
    + WALKING [1.5]
* TN
    + INVALID [1]
#### Confusion Matrix

sensed mode = ['WALKING', 'NO_SENSED']

ground truth = ['WALKING']

    [1,1]

In [ ]:
test_trip = {
    'ss_timeline'  : [{'start_ts' : 0.5, 'end_ts' : 1, 'mode' : 'WALKING'}, {'start_ts' : 1, 'end_ts' : 1.5, 'mode' : 'CYCLING'}],
    'gts_timeline' : [{'start_ts' : 0, 'end_ts' : 2, 'mode' : 'WALKING'}]
}
test_BMM = {'WALKING' : 'WALKING', 'CYCLING' : "CYCLING", 'NO_SENSED' : 'INVALID', 'NO_GT' : 'INVALID', 'INVALID' : 'INVALID'}

In [ ]:
res = get_binary_class_in_sec(..., ..., ..., test_BMM, test=True, test_trip=test_trip)

In [ ]:
res

In [ ]:
assert res[0]['WALKING'] == 0.5
assert res[1]['INVALID'] == 1.0 and res[1]['CYCLING'] == 0.5
assert res[2]['WALKING'] == 1.5
assert res[3]['INVALID'] == 1.0

In [ ]:
res = get_confusion_matrix(..., ..., ..., test=True, test_trip=test_trip)
df = pd.DataFrame(res).groupby('sensed_mode').sum()

In [ ]:
assert df.loc['WALKING'].loc['WALKING'] == 0.5
assert df.loc['CYCLING'].loc['WALKING'] == 0.5
assert df.loc['NO_SENSED'].loc['WALKING'] == 1

### No ground truth at beggining and end, unimodal

* pipeline timeline
WALKING (0, 2)

* ground truth timeline
WALKING (0.5, 1.5)

#### Multiclass classifiers
* TP
    + WALKING [1]
* FP
    + WALKING [1]
* FN
    + INVALID [1]
* TN
    + INVALID [1]
* $F_1$ score
    + ...
#### Confusion Matrix

sensed mode = ['WALKING']

ground truth = ['WALKING', 'NO_GT]

    [1,1]

In [ ]:
test_trip = {
    'gts_timeline'  : [{'start_ts' : 0.5, 'end_ts' : 1.5, 'mode' : 'WALKING'}],
    'ss_timeline' : [{'start_ts' : 0, 'end_ts' : 2, 'mode' : 'WALKING'}]
}
test_BMM = {'WALKING' : 'WALKING', 'NO_SENSED' : 'INVALID', 'NO_GT' : 'INVALID', 'INVALID' : 'INVALID'}

In [ ]:
res = get_binary_class_in_sec(..., ..., ..., test_BMM, test=True, test_trip=test_trip)

In [ ]:
assert res[0]['WALKING'] == 1
assert res[1]['WALKING'] == 1
assert res[2]['INVALID'] == 1
assert res[3]['INVALID'] == 1

In [ ]:
res = get_confusion_matrix(..., ..., ..., test=True, test_trip=test_trip)
df = pd.DataFrame(res).groupby('sensed_mode').sum()

In [ ]:
assert df.loc['WALKING'].loc['WALKING'] == 1
assert df.loc['WALKING'].loc['NO_GT'] == 1